# Домашна задача 3 - Земји
## Виртуелни општества 2018/2019
### Андреј Јанчевски - 151003

In [3]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
from matplotlib.colors import ListedColormap
import networkx as nx
import pandas
from collections import OrderedDict
import sys
sys.path.append('using_networkx')
from community_jolleycraig import detect_communities, split_communities, new_draw_nodes

In [4]:
dataset = pandas.read_csv("filtered_graph_data.csv", "\t", encoding="utf-8")
nrows, ncols = dataset.shape

In [5]:
countries_num_papers = OrderedDict()
countries_joint_papers = OrderedDict()

for i in range(0, nrows):
    paper = dataset.loc[i]
    countries = sorted(set(paper["countries"].split(';')))
    num_countries = len(countries)
    for j in range(0, num_countries):
        country = countries[j]
        if country != "Unknown":
            countries_num_papers.setdefault(country, 0)
            countries_num_papers[country] += 1
    for j in range(0, num_countries - 1):
        country = countries[j]
        for k in range(j + 1, num_countries):
            second_country = countries[k]
            pair = (country, second_country)
            countries_joint_papers.setdefault(pair, 0)
            countries_joint_papers[pair] += 1

In [6]:
percentile = 85
node_threshold = int(np.percentile(list(countries_num_papers.values()), percentile))
edge_threshold = int(np.percentile(list(countries_joint_papers.values()), percentile))
country_graph = nx.Graph()

selected_countries = set()
for country, num_papers in countries_num_papers.items():
    if num_papers > node_threshold:
        country_graph.add_node(country, weight=num_papers)
        selected_countries.add(country)
for pair, num_papers in countries_joint_papers.items():
    if num_papers > edge_threshold and pair[0] in selected_countries and pair[1] in selected_countries:
        country_graph.add_edge(pair[0], pair[1], weight=num_papers)

In [12]:
split_communities(country_graph)
coms = set(country_graph.node[n]['Community'] for n in country_graph.nodes())
print('Communities:', coms)

KeyError: 0

In [11]:
sorted_nodes = sorted(
    country_graph.nodes(data=True), key=lambda x: x[1]['Community'])
dim = len(sorted_nodes)
matrix = np.zeros((dim, dim))
for i in range(dim):
    for j in range(i + 1, dim):
        if country_graph.has_edge(sorted_nodes[i][0], sorted_nodes[j][0]):
            if sorted_nodes[i][1]['Community'] != sorted_nodes[j][1][
                    'Community']:
                matrix[i][j] = matrix[j][i] = 1
            else:
                matrix[i][j] = matrix[j][
                    i] = sorted_nodes[i][1]['Community'] + 2

cmap = ListedColormap(['whitesmoke'] + ['navy'] + color_seq)
plt.figure(1, figsize=(9, 9), dpi=1080 / 9)
plt.title("Country Graph Interaction Matrix")
plt.matshow(matrix, cmap=cmap)
plt.show()

KeyError: 'Community'